# Baseline Change Detection with Image Differencing

We're just going to subtract the old image from the new image and threshold to create a mask from the difference.

In [ ]:
import os
import numpy as np
import pandas as pd

import download

import rasterio
from rasterio.plot import show

### Set constants

In [ ]:
ITEM_TYPE = 'PSScene3Band'
START_DATE = "2020-06-01T00:00:00.000Z"
END_DATE = "2020-07-01T00:00:00.000Z"
CLOUD_TOLERANCE = .01
CLEAR_PERCENT = 95


### Read CSV

In [ ]:
il_df = pd.read_csv('~/notebooks/home/il_06-11-20_ind.csv')

lat, lon = il_df.loc[8, ['lat', 'lon']] #using the first in the list
coords = download.gen_box_coords(lat, lon)

### Download Images

In [ ]:
old_img = download.retrieve_planet_image(coords, 'old.tif', start_date='2019-06-01T00:00:00.000Z', end_date='2019-07-01T00:00:00.000Z', item_type=ITEM_TYPE)
new_img = download.retrieve_planet_image(coords, 'new.tif', start_date='2020-06-01T00:00:00.000Z', end_date='2020-07-01T00:00:00.000Z', item_type=ITEM_TYPE)

### Helpers to display images

In [ ]:
def show_rgb(img_file):
    with rasterio.open(img_file) as src:
        r,g,b,n = src.read()

    rgb = np.stack((r,g,b), axis=0)
    show(rgb/rgb.max())
    
def show_gray(img_file):
    with rasterio.open(img_file) as src:
        g = src.read(1)
    show(g/g.max())

In [ ]:
show_rgb('old.tif')
show_rgb('new.tif')

### Diff Images

In [ ]:
old_img = rasterio.open('old.tif')
new_img = rasterio.open('new.tif')

old_arr = old_img.read(1)
new_arr = new_img.read(1)

diff = np.absolute(new_arr - old_arr)

show(diff)

### Mask based on pixel difference threshold

In [ ]:
cut = 100

mask = np.copy(diff)
mask[mask <= cut] = 0
mask[mask > cut] = 1

show(mask)